In [1]:
!rm -vrf '/content/tobit_loss'

removed '/content/tobit_loss/tobit_loss.py'
removed '/content/tobit_loss/LICENSE'
removed directory '/content/tobit_loss/.git/branches'
removed '/content/tobit_loss/.git/logs/refs/remotes/origin/HEAD'
removed directory '/content/tobit_loss/.git/logs/refs/remotes/origin'
removed directory '/content/tobit_loss/.git/logs/refs/remotes'
removed '/content/tobit_loss/.git/logs/refs/heads/master'
removed directory '/content/tobit_loss/.git/logs/refs/heads'
removed directory '/content/tobit_loss/.git/logs/refs'
removed '/content/tobit_loss/.git/logs/HEAD'
removed directory '/content/tobit_loss/.git/logs'
removed '/content/tobit_loss/.git/index'
removed '/content/tobit_loss/.git/packed-refs'
removed '/content/tobit_loss/.git/hooks/pre-receive.sample'
removed '/content/tobit_loss/.git/hooks/pre-rebase.sample'
removed '/content/tobit_loss/.git/hooks/prepare-commit-msg.sample'
removed '/content/tobit_loss/.git/hooks/applypatch-msg.sample'
removed '/content/tobit_loss/.git/hooks/pre-applypatch.sampl

In [2]:
!git clone https://github.com/vlad-danaila/tobit_loss.git

Cloning into 'tobit_loss'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 71 (delta 37), reused 40 (delta 16), pack-reused 0
Unpacking objects: 100% (71/71), done.


In [3]:
import sys
sys.path.insert(0, '/content/tobit_loss')
from deep_tobit.normal_cumulative_distribution_function import cdf

In [5]:
from deep_tobit.util import to_torch, to_numpy, normalize
import torch as t
from scipy.stats import norm
import unittest
import numpy as np
from deep_tobit.normal_cumulative_distribution_function import cdf
from numpy.testing import assert_almost_equal

input = [10, 15, 20, 25, 30]

# manual gradient computing
x = np.array(input)
mean, std = x.mean(), x.std()
x_normalized = normalize(x, mean, std)
expected_cdf = norm.cdf(x_normalized)
expected_log_likelihood = np.log(expected_cdf)
expected_grad_log_likelihood_by_x = norm.pdf(x_normalized) / (expected_cdf * std)

# automatic gradient computing
x = to_torch(input, grad = True, device = 'cuda:0')
# in this test mean & std are considered constants
x_normalized = normalize(x, mean, std)
cdf_result = cdf(x_normalized)
assert_almost_equal(to_numpy(cdf_result), expected_cdf)

log_likelihood_result = t.log(cdf_result)
assert_almost_equal(to_numpy(log_likelihood_result), expected_log_likelihood)

loss = t.sum(log_likelihood_result)
loss.backward()

assert_almost_equal(to_numpy(x.grad), expected_grad_log_likelihood_by_x)